# Probabilistic Security Constrained Optimal Power FLow (pSCOPF) <br> using Monte Carlo with Acceleration

1. Simulate random data from weibul distribution for wind generation and lognormal for photovoltaic
    - Covariance of active power and power factors are input
    - Derive Reactive Power
    - Gaussian Copula
    - Assume constant load
2. Observe distribution for each line
    - Derive probability distribution parameters
    - Derive relationship between constant generation and distribution parameters
3. Create second copula for lines
4. Solve Optimal Power Flow using 2 additonal constraints
   - probability (total available power < demand) < available power reliability threshold
   - probability (line power > line threshold) < line reliability threshold

## IEEE 14 bus system

<img src="https://icseg.iti.illinois.edu/files/2013/10/WSCC14.png">

In [1]:
import copy
import numpy as np
import math
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas as pd
from pypower.api import case9, case14, runpf, runopf, printpf, ppoption, scale_load
from scipy.stats import ecdf, norm, beta, weibull_min, multivariate_normal as mvn
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [440]:
ppc = case14()
ppc2 = copy.deepcopy(ppc)
ppopt = ppoption(PF_ALG=2, PF_MAX_IT=10)
rpf = runpf(ppc, ppopt)

PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.

Converged in 0.05 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             14     Total Gen Capacity     772.4         -52.0 to 148.0
Generators         5     On-line Capacity       772.4         -52.0 to 148.0
Committed Gens     5     Generation (actual)    272.4              82.4
Loads             11     Load                   259.0              73.5
  Fixed           11       Fixed                259.0              73.5
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             1     Shunt (inj)             -0.0              21.2
Branches          20     Losses (I^2 * Z)        13.39             54.54

### Generate random numbers with consitent power triangle
For given means and standard deviations of P and power factor
calculate a set of random P and Qs
With no loss of generality the random disturbance is independent and normally distributed

In [422]:
def simulate(P, P_sd, pf_sd, N):
    P_hat = []
    Q_hat = []
    for a in range(N):
        rd_p = max(np.random.normal(P, P_sd),  0.01)
        rd_pf = min(np.random.normal(.925, pf_sd), .999)
        theta = math.acos(rd_pf)
                
        P_hat.append(rd_p)
        Q_hat.append(math.tan(theta)*rd_p)
    return P_hat, Q_hat

In [423]:
# convert covariancematrix to correlation matrix for numerical stability
def rho_matrix(X):
    Y = np.diagflat(np.sqrt(1/np.diag(X)))
    return np.matmul(Y, np.matmul(X, Y))

## Use a Gaussian Copula to simulate random behavior of 2 generators and associated power factors

In [424]:
cov = []
G = 2
for b in range(G*2):
    cov.append([np.random.uniform()*2 for a in range(G*2)])
cov = np.matmul(cov, np.transpose(cov))
cov = rho_matrix(cov)
for b in range(G*2):
    cov[b][b] = cov[b][b]+.1
print("Check if covariance is positive semi-definite: {}".format(np.linalg.det(cov)))
R = np.random.multivariate_normal([0]*G*2, cov, 120).T

Check if covariance is positive semi-definite: 0.06594102614011223


In [425]:
Renewable = [[math.exp(math.log(38)+x*.05) for x in R[0]]]       # photovoltaic, lognormal
Renewable.append(weibull_min.ppf(norm.cdf(R[1]), 3, .125)+.01)   # Wind, Weibull

DPF = []
B = []
for b in range(2, 4):
    PF = norm.cdf(R[b], 0, cov[b][b])*.1+.85
    DPF.append(PF)
    theta = [math.acos(pf) for pf in PF]
    B.append(theta)
    

### Apply Random disturbance to Bus 2, 3
Returns P and Q for each branch

In [218]:
# convert random numbers to branch P and Q
def gen_to_branch_PQ(ppc, P, theta, branch_id):
    Ps = []
    Qs = []
    Rs = []
    # set branch 10 to inactive
    ppc['branch'][branch_id][10] = 0.0
    for a in range(len(P[0])):
        for b in range(2):
            ppc['gen'][b+1][1] = P[b][a]
            ppc['gen'][b+1][2] = math.tan(theta[b][a])*P[b][a]
        
        r = runpf(ppc, ppopt, 'logfile.txt')
        Rs.append(r)
        Ps.append([r[0]['branch'][x][13] for x in range(20)])
        Qs.append([r[0]['branch'][x][14] for x in range(20)])
        
    ppc['branch'][branch_id][10] = 1.0

    return Ps, Qs, Rs       

In [426]:
# calculate derivative of branch source of P and Q with respect to change in generation
def shock(gen_id, x):
    theta = math.acos(.9)
    ppc2 = copy.deepcopy(ppc)
    Ps = []
    Qs = []    
    ppc2['gen'][gen_id][1] = x
    ppc2['gen'][gen_id][2] = math.tan(theta)*x
    r = runpf(ppc2, ppopt, 'logfile.txt')
    return [r[0]['branch'][b][13] for b in range(20)], [r[0]['branch'][b][14] for b in range(20)]

G3_0_P, G3_0_Q  = shock(3, 0)
G3_1_P, G3_1_Q = shock(3, 1)
G4_0_P, G4_0_Q = shock(4, 0)
G4_1_P, G4_1_Q = shock(4, 1)

dP_3 = [up-down for up, down in zip(G3_1_P, G3_0_P)]
dP_4 = [up-down for up, down in zip(G4_1_P, G4_0_P)]
dQ_3 = [up-down for up, down in zip(G3_1_Q, G3_0_Q)]
dQ_4 = [up-down for up, down in zip(G4_1_Q, G4_0_Q)]



PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.


### Convert marginals of each branch to normal distributions for Gaussian Copula

In [427]:
# return mean of the branch and the covariance of the copula
# the mean of the copula is 0

def get_P_stats(Ps,  u):
    mu = []
    vcv = []
    new_PQ = []
    shock_up = []
    shock_down = []
    tPQ = np.transpose(Ps)
    for x in range(len(tPQ)):#PQ.columns[[False if y == u else True for y in range(20)]]:
        if x != u:
            F = ecdf(tPQ[x]) # same thing as ordinal/N
            new_PQ.append( norm.ppf(F.cdf.evaluate(tPQ[x])*.9999+.00005))
            m = tPQ[x].mean()
            shock_up.append(norm.ppf(F.cdf.evaluate(m+.5)*.9999+.00005))
            shock_down.append(norm.ppf(F.cdf.evaluate(m-.5)*.9999+.00005))
            mu.append(m)
        
    vcv = np.cov(new_PQ)
    
    return mu, vcv, shock_up, shock_down

## Get distribution of each branch by outage

- 19 equations because line 13 is excluded. <br>
- Each equation has 19 variables b/c the one with the outage is excluded.

In [428]:
line_mu = []
line_sigma = []
SUp = []
SDown = []
for i in range(20):
    if i != 13:
        Ps, Qs, Rs = gen_to_branch_PQ(ppc2, Renewable, B, i)
        PsT = np.transpose(Ps)
        mu, sigma, shock_up, shock_down = get_P_stats(Ps, i)
        line_mu.append(mu)
        line_sigma.append(sigma)
        SUp.append(shock_up)
        SDown.append(shock_down)

PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 9 P-iterations and 9 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 9 P-iterations and 9 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 9 P-iterations and 9 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 9 P-iterations and 9 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 9 P-iterations and 9 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 10 P-iterations and 9 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled p

### Calculate relationship between reliability and deterministic generation

In [435]:
# change in Z value wrt change in Branch * change in Branch wrt change in Generation =
# Change in Z wrt to change in Generation

dz_dx_3 = []
dz_dx_4 = []
for outage in range(19):
    T3 = []
    T4 = []
    for b in range(19):
        if b >= outage:
            branch = b
        else:
            branch = b + 1
        T3.append( (SUp[outage][b]-SDown[outage][b])*dP_3[branch]) 
        T4.append( (SUp[outage][b]-SDown[outage][b])*dP_4[branch]) 
    dz_dx_3.append(T3)
    dz_dx_4.append(T4)


# lines 8 - 12 only
# the first two are positive and the second three are negative

u = [np.inf]*2+threshold[0][10:13]
m = [0]*5
cov =  rho_matrix([s[8:13] for s in line_sigma[0][8:13]])
for a in range(len(cov)):
    cov[a][a] = cov[a][a]+.1
lwr_lmt = threshold[0][8:10]+[-np.inf]*3

### Estimate the active power at the slack bus as a linear function of the active power at 6 and 8

In [433]:
X = []
Y = []
Z = []
for g in [1, 10, 20, 50]:
    for g2 in [1, 10, 20, 50]:
        ppc2 = copy.deepcopy(ppc)
        theta = math.acos(.85)
        
        r = runpf(ppc2, ppopt, 'logfile.txt')
        
        ppc2['gen'][3][1] = g
        ppc2['gen'][3][2] = math.tan(theta) * g
                
        ppc2['gen'][4][1] = g2
        ppc2['gen'][4][2] = math.tan(theta) * g2
                
        r2 = runpf(ppc2, ppopt, 'logfile.txt')
        
        #print("  Branch:       {} {}".format(r2[0]['branch'][0][13], r2[0]['branch'][0][14]))
        Z.append(r[0]['gen'][0][1]- r2[0]['gen'][0][1])
        Y.append(g2)
        X.append(g)
        
lr = LinearRegression().fit(np.transpose([X, Y]), Z)

PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 7 P-iterations and 6 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 6 P-iterations and 5 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled power flow converged in 7 P-iterations and 6 Q-iterations.
PYPOWER Version 5.1.17, 02-Sept-2024 -- AC Power Flow (fast-decoupled, XB)


Fast-decoupled po

- Flow from 10, 11, 12
- The only one connected to 8 is 13 (in reverse)
- Decision is how much to add to 6 and 8

branch$_{i}$(p$_{j}$) = power from source at line *i* as a result of p$_{j}$ power generated a station *j*
<br>
P[branch$_{i}$(p$_{j}$)>T] = probability power from source at line *i* exceeds threshold *T*

In [437]:
class SCOPF:
    def __init__(self, config):
        self.config = config
        
    def probability_of_failure(self, x):
        return norm.cdf(np.sum([math.exp(i) for i in x]) - self.config['demand']) - 1 + self.config['generation_reliability']
        
    def generation_cost(self,x):
        S = 0
        
        for c, s in zip(self.config['generation_costs'], x):
            S = S + c[0] * math.exp(s)**2 + c[1] * math.exp(s) + c[2]
        slack_load = self.config['slack_estimator'].predict([x])[0]
        
        self.config['slack_gen_cost'][0] * math.exp(slack_load)**2 +\
        self.config['slack_gen_cost'][1] * math.exp(slack_load) + self.config['slack_gen_cost'][2]
        
        return S
        
    def probability_of_exceedance(self, weight): # this is actually, the probability of 1 - exceedance

        dz_dx = self.config['dz_dx'][0] * weight[0] + self.config['dz_dx'][1] * weight[1]
        
        S = mvn.cdf(self.config['upper_line_limit'],
                    dz_dx, 
                    self.config['cov'], 
                    lower_limit=self.config['lower_line_limit'])
        
        return 1 - S - self.config['line_reliability']

In [420]:
solution = minimize(ieee14.generation_cost, x0=(0, 0), method='COBYLA', 
                    options={'maxiter': 10},
                    constraints=([{'type': 'ineq', 'fun': ieee14.probability_of_failure},
                                  {'type': 'ineq', 'fun': ieee14.probability_of_exceedance}]))


In [409]:
cov = [s[8:13] for s in  line_sigma[2][8:13]]
ieee14 = SCOPF({'demand': 200,
                'generation_reliability': .05,
                'generation_costs': [[.5, 3, 1], [.8, 2, 3]],   # coefficients of quadratic function of cost for optimized bus only
                'slack_estimator': lr,
                'slack_gen_cost': [.5, 1, 1],
                'upper_line_limit': u,
                'dz_dx': [np.array(dz_dx_3[2][8:13]), np.array(dz_dx_4[2][8:13])],
                'cov': cov,
                'lower_line_limit': lwr_lmt,
                'line_reliability': .2}
              )

In [421]:
print(solution)
print("Cost                      {}".format(ieee14.generation_cost(solution.x)))
print("Probability of Failure    {}".format(ieee14.probability_of_failure(solution.x)))
print("Probability of Exceedance {}".format(ieee14.probability_of_exceedance(solution.x)))


 message: Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.
 success: False
  status: 4
     fun: 4.014574277283623
       x: [-6.083e+00 -5.558e+00]
    nfev: 20
   maxcv: 0.95
Cost                      4.014574277283623
Probability of Failure    -0.95
Probability of Exceedance 0.08953455011259132
